Source: https://catalog.data.gov/dataset/nyc-jobs

In [12]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.graph_objects as go

In [13]:
df = pd.read_csv("https://raw.githubusercontent.com/MollenFerneus/Portfolio-Projects/refs/heads/main/Jobs_NYC/Jobs_NYC_Postings.csv", index_col=0)
df.head()

,Agency,Posting Type,# Of Positions,Business Title,Civil Service Title,Title Classification,Title Code No,Level,Job Category,Full-Time/Part-Time indicator,...,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
Job ID,,,,,,,,,,,,,,,,,,,,,
596449,DEPT OF ENVIRONMENT PROTECTION,External,1,ACCOUNTABLE MANAGER,ASSOCIATE PROJECT MANAGER,Competitive-1,22427,03,"Engineering, Architecture, & Planning Public S...",F,...,NaN,To apply click Apply Now DEP is an equal oppo...,NaN,"Delaware Aqueduct Shaft 6 Field Office, Wappin...",NaN,New York City Residency is not required for th...,08/11/2023,NaN,08/11/2023,02/10/2025
632978,DEPT OF ENVIRONMENT PROTECTION,External,1,ACCOUNTABLE MANAGER,ASSOCIATE PROJECT MANAGER,Competitive-1,22427,01,"Engineering, Architecture, & Planning Public S...",F,...,NaN,NaN,NaN,NaN,NaN,New York City Residency is not required for th...,04/29/2024,NaN,04/29/2024,02/10/2025
648277,DEPARTMENT OF TRANSPORTATION,Internal,1,Gardener,GARDENER,Competitive-1,81310,01,Health Building Operations & Maintenance Publi...,F,...,Preference will be given to those candidates w...,All resumes are to be submitted electronically...,40 Hours Per Week,NaN,NaN,New York City residency is generally required ...,08/22/2024,NaN,08/22/2024,02/10/2025
679139,DEPT OF HEALTH/MENTAL HYGIENE,Internal,1,"Child Care Registrar, Bureau of Childcare",COMMUNITY ASSOCIATE,Non-Competitive-5,56057,00,Constituent Services & Community Programs Health,F,...,NaN,NaN,NaN,NaN,NaN,New York City residency is generally required ...,09/16/2024,16-SEP-2025,09/16/2024,02/10/2025
644090,DEPT OF HEALTH/MENTAL HYGIENE,Internal,3,Call Agent,CUSTOMER INFORMATION REP,Competitive-1,60888,01,Communications & Intergovernmental Affairs,F,...,NaN,NaN,NaN,NaN,NaN,New York City residency is generally required ...,01/22/2025,22-MAY-2025,01/22/2025,02/10/2025


<span style="background-color:yellow; font-size:20px;">Remove unecessary data</span>

In [16]:
df = df[["Civil Service Title", "Full-Time/Part-Time indicator", "Career Level", "Salary Range From", "Salary Range To", "Salary Frequency"]]

In [17]:
df.head()

,Civil Service Title,Full-Time/Part-Time indicator,Career Level,Salary Range From,Salary Range To,Salary Frequency
Job ID,,,,,,
596449,ASSOCIATE PROJECT MANAGER,F,Experienced (non-manager),98470.0,133496.0,Annual
632978,ASSOCIATE PROJECT MANAGER,F,Experienced (non-manager),74041.0,107227.0,Annual
648277,GARDENER,F,Experienced (non-manager),48860.0,70321.0,Annual
679139,COMMUNITY ASSOCIATE,F,Experienced (non-manager),43143.0,49615.0,Annual
644090,CUSTOMER INFORMATION REP,F,Experienced (non-manager),40882.0,51500.0,Annual


<span style="background-color:yellow; font-size:20px;">Converting Daily and Hourly data into Annual</span>

In [18]:
df.loc[df["Salary Frequency"] == "Hourly", "Salary Range From"] *= 40 * 52
df.loc[df["Salary Frequency"] == "Hourly", "Salary Range To"] *= 40 * 52

df.loc[df["Salary Frequency"] == "Daily", "Salary Range From"] *= 5 * 52
df.loc[df["Salary Frequency"] == "Daily", "Salary Range To"] *= 5 * 52

df["Salary Frequency"] = "Annual"

<span style="background-color:yellow; font-size:20px;">Fills the empty data where minimum salary range is 0</span>

In [19]:
df["Salary Range From"] = df.apply(
    lambda row: row["Salary Range To"] if row["Salary Range From"] == 0 else row["Salary Range From"], 
    axis=1
)


<span style="background-color:yellow; font-size:20px;">Groups salaries by Title, Experience and Employment type</span>

In [20]:
df_grouped = df.groupby(["Civil Service Title", "Career Level", "Full-Time/Part-Time indicator"]).agg({
    "Salary Range From": "min",
    "Salary Range To": "max"
}).reset_index()
df_grouped = df_grouped.sort_values(by="Salary Range From", ascending = True)

In [21]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Salary Range by Employment Type"),
    
    # Dropdown for Full-time or Part-time
    dcc.Dropdown(
        id="employment-type",
        options=[
            {"label": "Full-time", "value": "F"},
            {"label": "Part-time", "value": "P"}
        ],
        value="F",  # Default to Full-time
        clearable=False
    ),

    # Dropdown for Career Level
    dcc.Dropdown(
        id="career-level",
        options=[{"label": level, "value": level} for level in df_grouped["Career Level"].unique()],
        value=df_grouped["Career Level"].unique()[0],  # Default to first level
        clearable=False
    ),

    # New Dropdown for Biggest Discrepancies
    dcc.Dropdown(
        id="discrepancy-filter",
        options=[
            {"label": "All Jobs", "value": "all"},
            {"label": "Top 20 Biggest Salary Gaps", "value": "top"},
            {"label": "Bottom 20 Smallest Salary Gaps", "value": "bottom"}
        ],
        value="all",  # Default to showing all jobs
        clearable=False
    ),

    # Search bar for job title filtering
    dcc.Input(
        id="title-search",
        type="text",
        placeholder="Search for a job title...",
        debounce=True
    ),

    # Graph output
    dcc.Graph(id="salary-graph")
])

# Callback to update the graph
@app.callback(
    Output("salary-graph", "figure"),
    [Input("employment-type", "value"),
     Input("career-level", "value"),
     Input("discrepancy-filter", "value"),
     Input("title-search", "value")]
)
def update_graph(selected_type, selected_level, discrepancy_filter, search_query):
    # Filter data
    filtered_df = df_grouped[
        (df_grouped["Full-Time/Part-Time indicator"] == selected_type) &
        (df_grouped["Career Level"] == selected_level)
    ].copy()  # Copy to avoid warnings

    if search_query:
        filtered_df = filtered_df[filtered_df["Civil Service Title"].str.contains(search_query, case=False, na=False)]

    if filtered_df.empty:
        return go.Figure()

    # Calculate salary discrepancy
    filtered_df["Salary Gap"] = filtered_df["Salary Range To"] - filtered_df["Salary Range From"]

    # Apply discrepancy filter
    if discrepancy_filter == "top":
        filtered_df = filtered_df.nlargest(20, "Salary Gap").sort_values(by="Salary Range From", ascending=True)  # Top 20 biggest gaps
    elif discrepancy_filter == "bottom":
        filtered_df = filtered_df.nsmallest(20, "Salary Gap").sort_values(by="Salary Range From", ascending=True)  # Bottom 20 smallest gaps

    num_bars = len(filtered_df)
    height_per_bar = 35
    min_height = 500
    max_height = 2000
    graph_height = max(min_height, min(num_bars * height_per_bar, max_height))

    fig = go.Figure()

    fig.add_trace(go.Bar(
        y=filtered_df["Civil Service Title"] + " (" + filtered_df["Career Level"] + ")",
        x=filtered_df["Salary Range From"],
        name="Min Salary",
        marker_color="wheat",
        orientation="h",
        hovertext=[
            f"{title}<br>Min Salary: {int(min_sal):,}$".replace(",", " ")    
            for title, min_sal in zip(filtered_df["Civil Service Title"], filtered_df["Salary Range From"])
        ],
        hoverinfo="text" 
    ))

    fig.add_trace(go.Bar(
        y=filtered_df["Civil Service Title"] + " (" + filtered_df["Career Level"] + ")",
        x=filtered_df["Salary Range To"] - filtered_df["Salary Range From"],  
        base=filtered_df["Salary Range From"],  
        name="Max Salary",
        marker_color="darkred",
        orientation="h",
        hovertext=[
            f"{title}<br>Max Salary: {int(max_sal):,}$".replace(",", " ")   
            for title, max_sal in zip(filtered_df["Civil Service Title"], filtered_df["Salary Range To"])
        ],
        hoverinfo="text"
    ))

    fig.update_layout(
        title=f"Salary Range for {'Full-time' if selected_type == 'F' else 'Part-time'} Jobs ({selected_level})",
        xaxis_title="Annual Salary ($)",
        yaxis_title="Job Title & Career Level",
        height=graph_height,
        barmode="overlay",
        margin=dict(l=300, r=50, t=50, b=50),
        yaxis=dict(
            automargin=True,
            tickfont=dict(size=max(10, min(20, 2000 / num_bars)))
        )
    )
    
    return fig

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)
